In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import pickle
import copy
from sklearn.preprocessing import normalize

In [2]:
os.getcwd()

'/home/jtl/Documents/shenhao/sat_image_nhts/code'

In [8]:
stratified_train100k = pd.read_csv("stratified_train100k.csv")
stratified_test100k = pd.read_csv("stratified_test100k.csv")
print("Shape of the 100K Training Set is: ", stratified_train100k.shape)
print("Shape of the 100K Testing Set is: ", stratified_test100k.shape)
pd.set_option('display.max_columns', None)
stratified_train100k.head(10)

Shape of the 100K Training Set is:  (100000, 262)
Shape of the 100K Testing Set is:  (10000, 262)


,Unnamed: 0,CDIVMSAR,CENSUS_D,CENSUS_R,DRIVER,DRVRCNT,ENDTIME,EDUC,GASPRICE,HBHTNRNT,HBHUR,HBPPOPDN,HBRESDN,HHFAMINC,HHSIZE,HHSTATE,HHSTFIPS,HHVEHCNT,HH_CBSA,HH_HISP,HH_RACE,HOMEOWN,HOUSEID,HTEEMPDN,LIF_CYC,MSACAT,MSASIZE,NONHHCNT,NUMADLT,NUMONTRP,PERSONID,PRMACT,RAIL,R_AGE,R_AGE_IMP,R_SEX,R_SEX_IMP,STRTTIME,TDAYDATE,TDCASEID,TDWKND,TRAVDAY,TRIPPURP,TRPACCMP,TRPHHACC,TRPMILES,TRPTRANS,TRVLCMIN,URBAN,URBANSIZE,URBRUR,WHYFROM,WHYTO,WHYTRP1S,WHYTRP90,WORKER,WRKCOUNT,WTTRDFIN,ALT_16,ALT_23,ALT_45,BIKE4EX,BIKESHARE,BIKE_DFR,BIKE_GKP,BORNINUS,CARSHARE,CNTTDTR,CONDNIGH,CONDPUB,CONDRIDE,CONDRIVE,CONDSPEC,CONDTAX,CONDTRAV,DELIVER,DIARY,DISTTOSC17,DISTTOWK17,FLEXTIME,FRSTHM17,GCDWORK,GT1JBLWK,HEALTH,HHRESP,LPACT,MCUSED,MEDCOND,MEDCOND6,NBIKETRP,NOCONG,NWALKTRP,OCCAT,PAYPROF,PHYACT,PTUSED,RIDESHARE,SAMEPLC,SCHTRN1,SCHTRN2,SCHTYP,TIMETOWK,VPACT,WALK4EX,WALK_DEF,WALK_GKQ,WKFMHMXX,WKFTPT,WKRMHM,WKSTFIPS,WRKTIME,WRKTRANS,WRK_HOME,WTPERFIN,YEARMILE,YRTOUS,BIKE,BIKE2SAVE,BUS,CAR,CNTTDHH,PARA,PC,PLACE,PRICE,PTRANS,SAMPSTRAT,SPHONE,TAB,TAXI,TRAIN,WALK,WALK2SAVE,WEBUSE17,WTHHFIN,YOUNGCHILD,CENSUS_R_1,CENSUS_R_2,CENSUS_R_3,CENSUS_R_4,ENDTIME_MORNING,ENDTIME_MORNING_PEAK,ENDTIME_NOON,ENDTIME_AFTERNOON,ENDTIME_EVENING_PEAK,ENDTIME_EVENING,EDUC_1,EDUC_2,EDUC_3,EDUC_4,EDUC_5,HBHTNRNT_NEW,HBHUR_C,HBHUR_R,HBHUR_S,HBHUR_T,HBHUR_U,HHFAMINC_NEW,HH_RACE_WHITE,HH_RACE_BLACK,HH_RACE_ASIAN,HH_RACE_MULTIPLE,HH_RACE_OTHER,HOMEOWN_OWN,HOMEOWN_RENT,ADULT_NO_CHI,ADULT_CHI_15,ADULT_CHI_21,ADULT_RET_NO_CHI,MSACAT_1,MSACAT_2,MSACAT_3,MSACAT_4,MSASIZE_NEW,PRMACT_WORK,PRMACT_ABSENSE,PRMACT_UNEMP,PRMACT_HOMEMAKER,PRMACT_SCHOOL,PRMACT_RETIRED,PRMACT_OTHER,RAIL_NEW,R_SEX_IMP_NEW,STRTTIME_MORNING,STRTTIME_MORNING_PEAK,STRTTIME_NOON,STRTTIME_AFTERNOON,STRTTIME_EVENING_PEAK,STRTTIME_EVENING,TDAYDATE_SPRING,TDAYDATE_SUMMER,TDAYDATE_AUTUMN,TDAYDATE_WINTER,TRAVDAY_SUN,TRAVDAY_MON,TRAVDAY_TUE,TRAVDAY_WED,TRAVDAY_THU,TRAVDAY_FRI,TRAVDAY_SAT,-9,TRIPPURP_HBO,TRIPPURP_HBSHOP,TRIPPURP_HBSOCREC,TRIPPURP_HBW,TRIPPURP_NHB,URBAN_1,URBAN_2,URBAN_3,URBAN_4,URBANSIZE_NEW,URBRUR_NEW,WHYTRP1S_HOME,WHYTRP1S_WORK,WHYTRP1S_SCHOOL,WHYTRP1S_MEDICAL,WHYTRP1S_SHOP,WHYTRP1S_SOCIAL,WHYTRP1S_PICK,WHYTRP1S_MEAL,WHYTRP1S_OTHER,WORKER_NEW,BORNINUS_NEW,CONDNIGH_NEW,CONDPUB_NEW,CONDRIDE_NEW,CONDRIVE_NEW,CONDSPEC_NEW,CONDTAX_NEW,CONDTRAV_NEW,DIARY_NEW,FLEXTIME_NEW,FRSTHM17_NEW,GT1JBLWK_NEW,MEDCOND_NEW,MEDCOND6_NEW,OCCAT_SERVICE,OCCAT_CLERICAL,OCCAT_MANU,OCCAT_RPOF,OCCAT_OTHER,PAYPROF_NEW,SCHTYP_PUBLIC,WKFTPT_FULL,WKFTPT_PART,BIKE_HIGH,BUS_HIGH,CAR_HIGH,PARA_HIGH,PC_HIGH,PLACE_AGREE,PRICE_AGREE,PTRANS_AGREE,SPHONE_HIGH,TAB_HIGH,TAXI_HIGH,WALK_HIGH,WALK2SAVE_AGREE,WEBUSE17_HIGH,HHCNTYFP,HHCT,CENSUS_ID
0,229701,53,5,3,1,2,1225,4,232.8,20,T,750,300,5,5,NC,37,2,XXXXX,2,1,1,30300425,150,4,3,1,0,2,1,2,1,2,29,29,1,1,1223,201609,303004250204,2,3,4,0,0,0.601,4,2,4,6,2,4,3,10,1,1,1,38340.844938,4,1,3,-1,-1,-1,-1,1,0,5,-1,-1,-1,-1,-1,-1,-1,2,1,-9.00,10.67,1,1,7.72,1,2,1,0,0,2,-1,0,11,8,4,-1,3,0,0,-1,-1,-1,-1,14,3,2,7,5,3,1,1,37,09:00 AM,3,2,105.043411,15000,-1,3,1,4,1,8,5,1,3,2,4,3,1,5,5,4,1,2,1,97.750732,3,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,20.0,0,0,0,1,0,42500,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,125000,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,127,11101,37127011101
1,610768,52,5,3,1,2,1515,5,236.9,5,T,300,300,7,2,NC,37,2,39580,2,1,1,40408278,350,2,2,4,0,2,2,1,1,2,27,27,1,1,1500,201703,404082780103,1,7,1,1,1,4.489,2,15,4,6,2,11,1,1,10,1,2,468739.580744,4,2,4,-1,-1,-1,-1,1,1,5,-1,-1,-1,-1,-1,-1,-1,10,1,-9.00,30.91,2,1,21.50,2,1,1,0,0,2,-1,0,30,5,4,-1,3,0,1,-1,-1,-1,-1,30,4,5,7,3,-1,1,2,37,07:15 AM,3,2,1284.218029,30000,-1,5,4,5,1,12,5,1,4,4,5,2,1,1,4,5,5,4,1,992.927607,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,10.0,0,0,0,1,0,87500,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,2000000,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,

In [10]:
print("HHFAMINC" in stratified_train100k.columns)
print("HHFAMINC_NEW" in stratified_train100k.columns)

True
True


In [11]:
#If the variable is continuous by nature and if categories are greater than 5, then treat them as numerical
categorical_variables = ["HHFAMINC", "R_SEX_IMP", "EDUC", "HH_RACE", "LIF_CYC", "HOMEOWN", "HBHUR",
                        "MSACAT", "CENSUS_R", "URBAN", "RAIL", "HBHTNRNT", "HBPPOPDN", "HBRESDN"]
#add HHSIZE, ADULT, as numerical variables
num_variables = ["R_AGE", "DRVRCNT", "HHSIZE"]
targets = ["TRPTRANS", "TRIPPURP", "HHVEHCNT"]
#add index
all_variables = categorical_variables + num_variables + targets + ["CENSUS_ID"]
print(len(all_variables))

21


In [12]:
#make deep copies
stratified_train100k = stratified_train100k[all_variables].copy(deep=True)
stratified_test100k = stratified_test100k[all_variables].copy(deep=True)

In [13]:
# 
print(stratified_train100k.head(10))
print(stratified_train100k.shape)

   HHFAMINC  R_SEX_IMP  EDUC  HH_RACE  LIF_CYC  HOMEOWN HBHUR  MSACAT  \
0         5          1     4        1        4        1     T       3   
1         7          1     5        1        2        1     T       2   
2         7          1     1        1        8        1     T       3   
3         7          1     5        1       10        1     T       4   
4         6          1     5        1       10        1     C       2   
5         6          2     5        1        2        2     T       4   
6         4          2     2        1       10        1     C       3   
7         9          2     4        1        4        1     C       2   
8         5          1     3        1       10        1     R       3   
9        11          1     2        1        8        1     T       2   

   CENSUS_R  URBAN  RAIL  HBHTNRNT  HBPPOPDN  HBRESDN  R_AGE  DRVRCNT  HHSIZE  \
0         3      4     2        20       750      300     29        2       5   
1         3      4     2         5

## Removing negative values (replace with 0 indicating missing)

In [14]:
remove_negatives = ["HHFAMINC", "EDUC", "HH_RACE", "LIF_CYC", "HOMEOWN", "HBHTNRNT", "HBPPOPDN", "HBRESDN"]

In [15]:
stratified_train100k[categorical_variables].values

array([[5, 1, 4, ..., 20, 750, 300],
       [7, 1, 5, ..., 5, 300, 300],
       [7, 1, 1, ..., 40, 300, 300],
       ...,
       [6, 2, 3, ..., 20, 50, 50],
       [10, 1, 5, ..., 20, 300, 300],
       [7, 1, 3, ..., 0, 750, 300]], dtype=object)

In [16]:
for elt in remove_negatives:
    stratified_train100k[elt].values[stratified_train100k[elt]<0] = 0
    stratified_test100k[elt].values[stratified_test100k[elt]<0] = 0

In [17]:
stratified_train100k.head()

,HHFAMINC,R_SEX_IMP,EDUC,HH_RACE,LIF_CYC,HOMEOWN,HBHUR,MSACAT,CENSUS_R,URBAN,RAIL,HBHTNRNT,HBPPOPDN,HBRESDN,R_AGE,DRVRCNT,HHSIZE,TRPTRANS,TRIPPURP,HHVEHCNT,CENSUS_ID
0,5,1,4,1,4,1,T,3,3,4,2,20,750,300,29,2,5,4,4,2,37127011101
1,7,1,5,1,2,1,T,2,3,4,2,5,300,300,27,2,2,2,1,2,37183053420
2,7,1,1,1,8,1,T,3,3,4,2,40,300,300,16,3,4,1,0,2,37191000601
3,7,1,5,1,10,1,T,4,3,2,2,5,750,300,74,1,2,0,4,1,37105030501
4,6,1,5,1,10,1,C,2,3,1,2,5,7000,1500,72,2,2,2,4,2,37183053422


In [19]:
print(stratified_train100k.shape)

(100000, 21)


## One hot encoding categorical variables

In [20]:
getEncoderListTrain = stratified_train100k.copy(deep=True)
encoderListTrain = set(pd.get_dummies(data=getEncoderListTrain[all_variables], columns=categorical_variables).columns)
print(len(encoderListTrain))

91


In [21]:
getEncoderListTest = stratified_test100k.copy(deep=True)
encoderListTest = set(pd.get_dummies(data=getEncoderListTest[all_variables], columns=categorical_variables).columns)
print(len(encoderListTest))

91


In [22]:
encoderListAll = list(encoderListTrain.union(encoderListTest))
encoderListAll.sort()
print(encoderListAll)
print(len(encoderListAll))

['CENSUS_ID', 'CENSUS_R_1', 'CENSUS_R_2', 'CENSUS_R_3', 'CENSUS_R_4', 'DRVRCNT', 'EDUC_1', 'EDUC_2', 'EDUC_3', 'EDUC_4', 'EDUC_5', 'HBHTNRNT_0', 'HBHTNRNT_20', 'HBHTNRNT_30', 'HBHTNRNT_40', 'HBHTNRNT_5', 'HBHTNRNT_50', 'HBHTNRNT_60', 'HBHTNRNT_70', 'HBHTNRNT_80', 'HBHTNRNT_90', 'HBHTNRNT_95', 'HBHUR_C', 'HBHUR_R', 'HBHUR_S', 'HBHUR_T', 'HBHUR_U', 'HBPPOPDN_1500', 'HBPPOPDN_17000', 'HBPPOPDN_300', 'HBPPOPDN_3000', 'HBPPOPDN_30000', 'HBPPOPDN_50', 'HBPPOPDN_7000', 'HBPPOPDN_750', 'HBRESDN_1500', 'HBRESDN_17000', 'HBRESDN_300', 'HBRESDN_3000', 'HBRESDN_30000', 'HBRESDN_50', 'HBRESDN_7000', 'HBRESDN_750', 'HHFAMINC_1', 'HHFAMINC_10', 'HHFAMINC_11', 'HHFAMINC_2', 'HHFAMINC_3', 'HHFAMINC_4', 'HHFAMINC_5', 'HHFAMINC_6', 'HHFAMINC_7', 'HHFAMINC_8', 'HHFAMINC_9', 'HHSIZE', 'HHVEHCNT', 'HH_RACE_1', 'HH_RACE_2', 'HH_RACE_3', 'HH_RACE_4', 'HH_RACE_5', 'HH_RACE_6', 'HH_RACE_97', 'HOMEOWN_1', 'HOMEOWN_2', 'HOMEOWN_97', 'LIF_CYC_1', 'LIF_CYC_10', 'LIF_CYC_2', 'LIF_CYC_3', 'LIF_CYC_4', 'LIF_CYC_5', 'L

In [23]:
def one_hot_encode(variables, data_frame):
    data = data_frame.copy()
    #create columns for the indicator variables
    one_hot_encoding = pd.get_dummies(data = data, columns=variables)
    one_hot_encoding = one_hot_encoding.reindex(columns=encoderListAll, fill_value = 0)    
    return one_hot_encoding

In [24]:
encoded_train100k = one_hot_encode(categorical_variables, stratified_train100k)
print(encoded_train100k.shape)

(100000, 91)


In [25]:
encoded_test100k = one_hot_encode(categorical_variables, stratified_test100k)
print(encoded_test100k.shape)

(10000, 91)


In [26]:
encoded_train100k.head(100)

,CENSUS_ID,CENSUS_R_1,CENSUS_R_2,CENSUS_R_3,CENSUS_R_4,DRVRCNT,EDUC_1,EDUC_2,EDUC_3,EDUC_4,EDUC_5,HBHTNRNT_0,HBHTNRNT_20,HBHTNRNT_30,HBHTNRNT_40,HBHTNRNT_5,HBHTNRNT_50,HBHTNRNT_60,HBHTNRNT_70,HBHTNRNT_80,HBHTNRNT_90,HBHTNRNT_95,HBHUR_C,HBHUR_R,HBHUR_S,HBHUR_T,HBHUR_U,HBPPOPDN_1500,HBPPOPDN_17000,HBPPOPDN_300,HBPPOPDN_3000,HBPPOPDN_30000,HBPPOPDN_50,HBPPOPDN_7000,HBPPOPDN_750,HBRESDN_1500,HBRESDN_17000,HBRESDN_300,HBRESDN_3000,HBRESDN_30000,HBRESDN_50,HBRESDN_7000,HBRESDN_750,HHFAMINC_1,HHFAMINC_10,HHFAMINC_11,HHFAMINC_2,HHFAMINC_3,HHFAMINC_4,HHFAMINC_5,HHFAMINC_6,HHFAMINC_7,HHFAMINC_8,HHFAMINC_9,HHSIZE,HHVEHCNT,HH_RACE_1,HH_RACE_2,HH_RACE_3,HH_RACE_4,HH_RACE_5,HH_RACE_6,HH_RACE_97,HOMEOWN_1,HOMEOWN_2,HOMEOWN_97,LIF_CYC_1,LIF_CYC_10,LIF_CYC_2,LIF_CYC_3,LIF_CYC_4,LIF_CYC_5,LIF_CYC_6,LIF_CYC_7,LIF_CYC_8,LIF_CYC_9,MSACAT_1,MSACAT_2,MSACAT_3,MSACAT_4,RAIL_1,RAIL_2,R_AGE,R_SEX_IMP_1,R_SEX_IMP_2,TRIPPURP,TRPTRANS,URBAN_1,URBAN_2,URBAN_3,URBAN_4
0,37127011101,0,0,1,0,2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,29,1,0,4,4,0,0,0,1
1,37183053420,0,0,1,0,2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,2,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,27,1,0,1,2,0,0,0,1
2,37191000601,0,0,1,0,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,16,1,0,0,1,0,0,0,1
3,37105030501,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,74,1,0,4,0,0,1,0,0
4,37183053422,0,0,1,0,2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,2,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,72,1,0,4,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,37183053724,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,80,1,0,1,2,1,0,0,0
96,37051003203,0,0,1,0,2,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,2,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,68,0,1,1,2,1,0,0,0
97,37147002001,0,0,1,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,2,3,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,63,0,1,1,3,0,0,0,1
98,37181960100,0,0,1,0,2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,73,0,1,2,1,0,0,0,1


## Normalizing continuous variables

In [27]:
encoded_train100k[num_variables]

,R_AGE,DRVRCNT,HHSIZE
0,29,2,5
1,27,2,2
2,16,3,4
3,74,1,2
4,72,2,2
...,...,...,...
99995,54,2,2
99996,54,2,4
99997,66,2,2
99998,36,2,4


In [28]:
from sklearn.preprocessing import MinMaxScaler
for num_variable in num_variables:
    scaler = MinMaxScaler()
    single_normalized = scaler.fit_transform(encoded_train100k.loc[:,num_variable].values.reshape(-1,1).astype(np.float32))
    
    encoded_train100k.loc[:,num_variable] = single_normalized


In [29]:
encoded_train100k[num_variables]

,R_AGE,DRVRCNT,HHSIZE
0,0.37,0.222222,0.333333
1,0.35,0.222222,0.083333
2,0.24,0.333333,0.250000
3,0.82,0.111111,0.083333
4,0.80,0.222222,0.083333
...,...,...,...
99995,0.62,0.222222,0.083333
99996,0.62,0.222222,0.250000
99997,0.74,0.222222,0.083333
99998,0.44,0.222222,0.250000


In [30]:
for num_variable in num_variables:
    scaler = MinMaxScaler()
    single_normalized = scaler.fit_transform(encoded_test100k.loc[:,num_variable].values.reshape(-1,1).astype(np.float32))
    
    encoded_test100k.loc[:,num_variable] = single_normalized

In [31]:
encoded_test100k[num_variables]

,R_AGE,DRVRCNT,HHSIZE
0,0.515152,0.333333,0.363636
1,0.717172,0.333333,0.090909
2,0.585859,0.333333,0.090909
3,0.676768,0.333333,0.090909
4,0.707071,0.333333,0.090909
...,...,...,...
9995,0.424242,0.333333,0.181818
9996,0.707071,0.333333,0.090909
9997,0.242424,0.666667,0.454545
9998,0.636364,0.333333,0.090909


In [41]:
print(encoded_train100k.shape)
print(encoded_test100k.shape)

(100000, 91)
(10000, 91)


In [46]:
y_train_hhvehcnt100k = encoded_train100k["HHVEHCNT"].copy(deep=True).values.reshape(-1,1)
y_train_trippurp100k = encoded_train100k["TRIPPURP"].copy(deep=True).values.reshape(-1,1)
y_train_trptrans100k = encoded_train100k["TRPTRANS"].copy(deep=True).values.reshape(-1,1)

y_test_hhvehcnt100k = encoded_test100k["HHVEHCNT"].copy(deep=True).values.reshape(-1,1)
y_test_trippurp100k = encoded_test100k["TRIPPURP"].copy(deep=True).values.reshape(-1,1)
y_test_trptrans100k = encoded_test100k["TRPTRANS"].copy(deep=True).values.reshape(-1,1)

x_train100k = encoded_train100k.drop(columns = targets,inplace=False)
x_test100k = encoded_test100k.drop(columns = targets,inplace=False)

In [47]:
print(targets)
print(y_train_hhvehcnt100k.shape)
print(y_train_trippurp100k.shape)
print(y_train_trptrans100k.shape)
print(x_train100k.shape)
print(x_test100k.shape)

['TRPTRANS', 'TRIPPURP', 'HHVEHCNT']
(100000, 1)
(100000, 1)
(100000, 1)
(100000, 88)
(10000, 88)


In [48]:
x_train100k.head(10)

,CENSUS_ID,CENSUS_R_1,CENSUS_R_2,CENSUS_R_3,CENSUS_R_4,DRVRCNT,EDUC_1,EDUC_2,EDUC_3,EDUC_4,EDUC_5,HBHTNRNT_0,HBHTNRNT_20,HBHTNRNT_30,HBHTNRNT_40,HBHTNRNT_5,HBHTNRNT_50,HBHTNRNT_60,HBHTNRNT_70,HBHTNRNT_80,HBHTNRNT_90,HBHTNRNT_95,HBHUR_C,HBHUR_R,HBHUR_S,HBHUR_T,HBHUR_U,HBPPOPDN_1500,HBPPOPDN_17000,HBPPOPDN_300,HBPPOPDN_3000,HBPPOPDN_30000,HBPPOPDN_50,HBPPOPDN_7000,HBPPOPDN_750,HBRESDN_1500,HBRESDN_17000,HBRESDN_300,HBRESDN_3000,HBRESDN_30000,HBRESDN_50,HBRESDN_7000,HBRESDN_750,HHFAMINC_1,HHFAMINC_10,HHFAMINC_11,HHFAMINC_2,HHFAMINC_3,HHFAMINC_4,HHFAMINC_5,HHFAMINC_6,HHFAMINC_7,HHFAMINC_8,HHFAMINC_9,HHSIZE,HH_RACE_1,HH_RACE_2,HH_RACE_3,HH_RACE_4,HH_RACE_5,HH_RACE_6,HH_RACE_97,HOMEOWN_1,HOMEOWN_2,HOMEOWN_97,LIF_CYC_1,LIF_CYC_10,LIF_CYC_2,LIF_CYC_3,LIF_CYC_4,LIF_CYC_5,LIF_CYC_6,LIF_CYC_7,LIF_CYC_8,LIF_CYC_9,MSACAT_1,MSACAT_2,MSACAT_3,MSACAT_4,RAIL_1,RAIL_2,R_AGE,R_SEX_IMP_1,R_SEX_IMP_2,URBAN_1,URBAN_2,URBAN_3,URBAN_4
0,37127011101,0,0,1,0,0.222222,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.333333,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0.37,1,0,0,0,0,1
1,37183053420,0,0,1,0,0.222222,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.083333,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0.35,1,0,0,0,0,1
2,37191000601,0,0,1,0,0.333333,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.250000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0.24,1,0,0,0,0,1
3,37105030501,0,0,1,0,0.111111,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.083333,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0.82,1,0,0,1,0,0
4,37183053422,0,0,1,0,0.222222,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.083333,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.80,1,0,1,0,0,0
5,37189920602,0,0,1,0,0.222222,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.083333,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0.47,0,1,0,0,0,1
6,37081012612,0,0,1,0,0.222222,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.083333,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.85,0,1,1,0,0,0
7,37119005517,0,0,1,0,0.222222,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.250000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0.43,0,1,1,0,0,0
8,37127011500,0,0,1,0,0.333333,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.166667,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.85,1,0,0,0,0,1
9,37119006304,0,0,1,0,0.666667,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.416667,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0.26,1,0,0,0,0,1


In [49]:
x_test100k.head(10)

,CENSUS_ID,CENSUS_R_1,CENSUS_R_2,CENSUS_R_3,CENSUS_R_4,DRVRCNT,EDUC_1,EDUC_2,EDUC_3,EDUC_4,EDUC_5,HBHTNRNT_0,HBHTNRNT_20,HBHTNRNT_30,HBHTNRNT_40,HBHTNRNT_5,HBHTNRNT_50,HBHTNRNT_60,HBHTNRNT_70,HBHTNRNT_80,HBHTNRNT_90,HBHTNRNT_95,HBHUR_C,HBHUR_R,HBHUR_S,HBHUR_T,HBHUR_U,HBPPOPDN_1500,HBPPOPDN_17000,HBPPOPDN_300,HBPPOPDN_3000,HBPPOPDN_30000,HBPPOPDN_50,HBPPOPDN_7000,HBPPOPDN_750,HBRESDN_1500,HBRESDN_17000,HBRESDN_300,HBRESDN_3000,HBRESDN_30000,HBRESDN_50,HBRESDN_7000,HBRESDN_750,HHFAMINC_1,HHFAMINC_10,HHFAMINC_11,HHFAMINC_2,HHFAMINC_3,HHFAMINC_4,HHFAMINC_5,HHFAMINC_6,HHFAMINC_7,HHFAMINC_8,HHFAMINC_9,HHSIZE,HH_RACE_1,HH_RACE_2,HH_RACE_3,HH_RACE_4,HH_RACE_5,HH_RACE_6,HH_RACE_97,HOMEOWN_1,HOMEOWN_2,HOMEOWN_97,LIF_CYC_1,LIF_CYC_10,LIF_CYC_2,LIF_CYC_3,LIF_CYC_4,LIF_CYC_5,LIF_CYC_6,LIF_CYC_7,LIF_CYC_8,LIF_CYC_9,MSACAT_1,MSACAT_2,MSACAT_3,MSACAT_4,RAIL_1,RAIL_2,R_AGE,R_SEX_IMP_1,R_SEX_IMP_2,URBAN_1,URBAN_2,URBAN_3,URBAN_4
0,37013930400,0,0,1,0,0.333333,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.363636,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0.515152,1,0,0,1,0,0
1,37193961200,0,0,1,0,0.333333,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.090909,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0.717172,0,1,0,0,0,1
2,37119006007,0,0,1,0,0.333333,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0.090909,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0.585859,0,1,1,0,0,0
3,37151030700,0,0,1,0,0.333333,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.090909,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0.676768,1,0,0,0,0,1
4,37105030601,0,0,1,0,0.333333,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.090909,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0.707071,1,0,0,1,0,0
5,37129010100,0,0,1,0,0.166667,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.000000,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.626263,0,1,1,0,0,0
6,37141920501,0,0,1,0,0.333333,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.090909,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.646465,0,1,0,0,0,1
7,37195000400,0,0,1,0,0.333333,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.090909,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0.727273,1,0,0,1,0,0
8,37159050500,0,0,1,0,0.166667,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0.000000,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.606061,1,0,1,0,0,0
9,37097061202,0,0,1,0,0.333333,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.363636,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0.434343,1,0,1,0,0,0


In [50]:
np.save("x_train100k", x_train100k)
np.save("y_train_hhvehcnt100k", y_train_hhvehcnt100k)
np.save("y_train_trippurp100k", y_train_trippurp100k)
np.save("y_train_trptrans100k", y_train_trptrans100k)

np.save("x_test100k", x_test100k)
np.save("y_test_hhvehcnt100k", y_test_hhvehcnt100k)
np.save("y_test_trippurp100k", y_test_trippurp100k)
np.save("y_test_trptrans100k", y_test_trptrans100k)

In [1]:
!pwd

/home/jtl/Documents/shenhao/sat_image_nhts/code
